# Piecewise natural cubic spline

<img src='xkcd_curve_fitting.png' style="float:center; width:500px" />

In [1]:
from datetime import datetime
from multiprocessing import Pool
from typing import List
import basis_expansions as bsx
import matplotlib.axes as axes
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [2]:
start_time = datetime.now()

In [3]:
# Data set must not contain NaN, inf, or -inf
file_name = 'predicted.csv'
figure_width_height = (6, 4)
x_axis_label = 'Abscissa'
y_axis_label = 'Ordinate'
axis_title = 'Piecewise natural cubic spline'
# Change this list to try different amounts of smoothing
num_knots = [
    10, 20, 30, 40, 50, 60, 70, 80, 90, 100,
    110, 120, 130, 140, 150, 160, 170, 180, 190, 200,
]
c = cm.Paired.colors
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [4]:
def main():
    data = pd.read_csv(file_name)
    global x, y, min_val, max_val
    x = data['abscissa']
    y = data['ordinate']
    min_val=min(x)
    max_val=max(x)
    with Pool() as pool:
        for _ in pool.imap_unordered(plot_cubic_thing, num_knots):
            pass

In [5]:
def plot_cubic_thing(numknots: List[int]) -> None:
    model = get_natural_cubic_spline(
        x, y, min_val, max_val, n_knots=numknots
    )
    fig = plt.figure(figsize=figure_width_height)
    ax = fig.add_subplot(111)
    ax.plot(x, y, ls='', marker='.', color=c[1], alpha=0.20)
    ax.plot(
        x, model.predict(x), marker='', color=c[5],
        label=f'number knots = {numknots}'
    )
    ax.legend(frameon=False, loc='best')
    ax.set_title(axis_title, fontweight='bold')
    ax.set_xlabel(x_axis_label, fontweight='bold')
    ax.set_ylabel(y_axis_label, fontweight='bold')
    despine(ax)
    ax.figure.savefig(f'natural_cubic_spline_{numknots}.svg', format='svg')

In [6]:
def despine(ax: axes.Axes) -> None:
    '''
    Remove the top and right spines of a graph.

    There is only one x axis, on the bottom, and one y axis, on the left.
    '''
    for spine in 'right', 'top':
        ax.spines[spine].set_visible(False)

In [7]:
def get_natural_cubic_spline(x, y, minval=None, maxval=None, n_knots=None, knots=None):
    '''
    Natural cubic spline model

    For the knots, give (a) `knots` (as an array) or
    (b) minval, maxval and n_knots.

    If the knots are not directly specified, the resulting knots are
    equally-spaced within the *interior* of (max, min).  That is,
    the endpoints are *not* included as knots.

    Parameters
    ----------
    x: np.array of float; the input data
    y: np.array of float; the outpur data
    minval: float; minimum of interval containing the knots.
    maxval: float ; maximum of the interval containing the knots.
    n_knots: positive integer; the number of knots to create.
    knots: array or list of floats; the knots.

    Returns
    --------
    model: a model object
        The returned model will have following method:
        - predict(x):
            x is a numpy array. This will return the predicted
            y-values.
    '''
    if knots:
        spline = bsx.NaturalCubicSpline(knots=knots)
    else:
        spline = bsx.NaturalCubicSpline(max=maxval, min=minval, n_knots=n_knots)

    p = Pipeline([
        ('nat_cubic', spline),
        ('regression', LinearRegression(fit_intercept=True))
    ])

    p.fit(x, y)

    return p

In [8]:
if __name__ == '__main__':
    main()

In [9]:
end_time = datetime.now()
round((end_time - start_time).total_seconds(), 3)

2.239

# References

- [Wikipedia Smoothing spline](https://en.wikipedia.org/wiki/Smoothing_spline)